<a href="https://colab.research.google.com/github/qw04/ageEstimation/blob/main/tryingOutNonCategoricalLabels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
import kaggle
! kaggle datasets download -d frabbisw/facial-age
! mkdir ageData1
! unzip facial-age.zip -d ageData1
! kaggle datasets download -d jangedoo/utkface-new
! mkdir ageData2
! unzip utkface-new.zip -d ageData2

Streaming output truncated to the last 5000 lines.
  inflating: ageData1/face_age/face_age/024/7366.png  
  inflating: ageData1/face_age/face_age/024/7406.png  
  inflating: ageData1/face_age/face_age/024/7411.png  
  inflating: ageData1/face_age/face_age/024/7497.png  
  inflating: ageData1/face_age/face_age/024/7511.png  
  inflating: ageData1/face_age/face_age/024/7520.png  
  inflating: ageData1/face_age/face_age/024/7558.png  
  inflating: ageData1/face_age/face_age/024/7600.png  
  inflating: ageData1/face_age/face_age/024/7629.png  
  inflating: ageData1/face_age/face_age/024/765.png  
  inflating: ageData1/face_age/face_age/024/7691.png  
  inflating: ageData1/face_age/face_age/024/7771.png  
  inflating: ageData1/face_age/face_age/024/7782.png  
  inflating: ageData1/face_age/face_age/024/7817.png  
  inflating: ageData1/face_age/face_age/024/786.png  
  inflating: ageData1/face_age/face_age/024/7955.png  
  inflating: ageData1/face_age/face_age/024/8099.png  
  inflating: age

In [ ]:
import math
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
data_dir1 = "/content/ageData1/face_age"
data_dir2 = "/content/ageData2/UTKFace"
EPOCHS = 20
IMG_WIDTH = 50
IMG_HEIGHT = 50
NUM_CATEGORIES = 1
TEST_SIZE = 0.3
images = []
labels = []

In [ ]:
another_useless_array = []
for folder in os.listdir(data_dir1):
  folder_path = os.path.join(data_dir1, folder)
  if os.path.isdir(folder_path) and folder != 'face_age':
    temp = int(folder)
    for file in os.listdir(folder_path):
      try:
        if folder != 'face_age':
          image = cv2.imread(os.path.join(folder_path, file), cv2.IMREAD_COLOR)
          image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_AREA)
          images.append(image)
          labels.append(temp)
      except Exception as e:
        print(e)
        another_useless_array.append(folder)
        pass
another_useless_set = set(another_useless_array)
print(another_useless_set)
#print(image[0])

set()


In [ ]:
for randomFile in os.listdir(data_dir2):
  folder_path = os.path.join(data_dir2, randomFile)
  var = randomFile[:2]
  if not var.isdigit():
    var = var[0]
  temp = int(var)
  try:  
    image = cv2.imread(folder_path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_AREA)
    images.append(image)
    labels.append(temp)
  except Exception as e:
    another_useless_array.append(randomFile)
    pass
print(set(another_useless_array))

set()


In [ ]:
#scale labels

In [ ]:
def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation="sigmoid", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation="sigmoid"),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES * 64, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES * 32, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES * 16, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES * 8, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES * 4, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES * 2, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax")
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [ ]:
model = get_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 48, 48, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 4, 4, 32)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dropout_14 (Dropout)        (None, 512)              

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    np.array(images), np.array(labels), test_size=TEST_SIZE
    )

In [ ]:
model.fit(x_train, y_train,
          epochs=EPOCHS,
          )

Epoch 1/20
214/214 [==============================] - 13s 57ms/step - loss: 0.0000e+00 - accuracy: 0.1124
Epoch 2/20
214/214 [==============================] - 12s 57ms/step - loss: 0.0000e+00 - accuracy: 0.1124
Epoch 3/20
214/214 [==============================] - 12s 57ms/step - loss: 0.0000e+00 - accuracy: 0.1124
Epoch 4/20
214/214 [==============================] - 12s 57ms/step - loss: 0.0000e+00 - accuracy: 0.1124
Epoch 5/20
214/214 [==============================] - 12s 57ms/step - loss: 0.0000e+00 - accuracy: 0.1124
Epoch 6/20
214/214 [==============================] - 12s 56ms/step - loss: 0.0000e+00 - accuracy: 0.1124
Epoch 7/20
214/214 [==============================] - 12s 56ms/step - loss: 0.0000e+00 - accuracy: 0.1124
Epoch 8/20
214/214 [==============================] - 12s 56ms/step - loss: 0.0000e+00 - accuracy: 0.1124
Epoch 9/20
214/214 [==============================] - 12s 57ms/step - loss: 0.0000e+00 - accuracy: 0.1124
Epoch 10/20
214/214 [=========================

In [ ]:
predictions = model.predict(x_test)
predictions = predictions.tolist()

In [ ]:
predictions[0]

[1.0]

In [ ]:
correct = 0
incorrect = 0
for i, j in zip(predictions, y_test):
  if j > 20:
    tempTest = 1
  else:
    tempTest = 0
  if i > 20:
    tempPrediction = 1
  else:
    tempPrediction = 0
  if tempPrediction == tempTest:
    correct += 1
  else:
    incorrect += 1
accuracy = (correct)/(correct+incorrect)
print(f'accuracy:{accuracy}')

TypeError: ignored

In [ ]:
print(set(predictions))

TypeError: ignored

In [ ]:
print(y_test[:5])

[50 41 26  7  2]


In [ ]:
print(set(labels))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 99, 100, 101, 110}
